In [5]:
!pip install keras-tuner

In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [7]:
fashion_mnist= keras.datasets.fashion_mnist

In [8]:
(train_images, train_labels),(test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [9]:
train_images = train_images/255.0
test_images = test_images/255.0

In [10]:
train_images[0].shape

(28, 28)

In [11]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step =16),
          kernel_size= hp.Choice('conv_1_kernel', values = [3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step =16),
          kernel_size= hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value = 32, max_value = 128, step =16),
           activation='relu',
      ),
      keras.layers.Dense(10, activation = 'softmax') #output layer
  ])
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values = [1e-2 , 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [14]:
tuner_search= RandomSearch(build_model,
                           objective='val_accuracy',
                           max_trials=5, directory='output', project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |96                |?                 
conv_1_kernel     |3                 |?                 
conv_2_filter     |48                |?                 
conv_2_kernel     |5                 |?                 
dense_1_units     |96                |?                 
learning_rate     |0.01              |?                 

Epoch 1/3
 719/1688 [===========>..................] - ETA: 4:40 - loss: 0.6424 - accuracy: 0.7849

In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()